In [1]:
import pickle

import numpy as np
import pandas as pd

from itertools import product, chain, combinations

from tqdm.notebook import tqdm

def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

In [2]:
df_classes = pd.read_csv('PI_classes.csv', index_col=0)

df_classes

,Bloco,Capacidade,Andar,Sala,Materia,Codigo,Turma,Horario,Oferecidas,Ocupadas
0,CTC 39 - Bloco A (etapa 1) - Sala de aula Sala...,45,ARQ05 - 1ºAndar,ARQ201,"Historia da Arte, Arquitetura e Urbanismo II",ARQ5622,02207A,2. 0910-3,25,20
1,CTC 39 - Bloco A (etapa 1) - Sala de aula Sala...,45,ARQ05 - 1ºAndar,ARQ201,"História da Arte, Arquitetura e Urbanismo III",ARQ5623,04207A,2. 1420-4,25,20
2,CTC 39 - Bloco A (etapa 1) - Sala de aula Sala...,45,ARQ05 - 1ºAndar,ARQ201,Arquitetura Latino Americana,ARQ5626,08207A,3. 0910-2,25,11
3,CTC 39 - Bloco A (etapa 1) - Sala de aula Sala...,45,ARQ05 - 1ºAndar,ARQ201,Criatividade e Inovação,EGC5027,09207,3. 1420-4,30,11
4,CTC 39 - Bloco A (etapa 1) - Sala de aula Sala...,45,ARQ05 - 1ºAndar,ARQ201,Análise Estrutural I,EMC5313,10203,4. 0730-3,20,10
...,...,...,...,...,...,...,...,...,...,...
1271,LIN Laboratório de Instrumentação Laboratório 16,16,NaN,LIN,Projeto Integrador,DAS5104,07220A,5. 1620-2,21,8
1272,LIN Laboratório de Instrumentação Laboratório 16,16,NaN,LIN,Instrumentação em Controle,DAS5151,06220B,6. 1010-2,15,10
1273,LPR Laboratório de Projetos Laboratório 16,16,NaN,LPR,Introdução à Engenharia de Controle e Automação,DAS5411,01220C,5. 1010-2,14,10
1274,LPR Laboratório de Projetos Laboratório 16,16,NaN,LPR,Introdução à Engenharia de Controle e Automação,DAS5411,01220A,5. 1330-2,14,12


In [3]:
df_rooms = pd.read_csv('PI_rooms.csv')
df_rooms

,Lugar,Sala,Carteiras,Características,Ar condicionado,Projetor,Laboratório
0,ARQ05 - 1ºAndar,ARQ201,45,"2 Ar condicionado, 1 Mesa do professor (sala d...",2,1,0
1,ARQ04 - 1ºAndar,ARQ202,40,"1 Ar condicionado, 1 Projetor multimídia, 1 Me...",1,1,0
2,ARQ03 - 1ºAndar,ARQ203,40,"2 Ar condicionado, 1 Projetor multimídia, 1 Me...",2,1,0
3,ARQ02 - Térreo,ARQ204,40,"2 Ar condicionado, 1 Projetor multimídia, 1 Qu...",2,1,0
4,ARQ09 - 2º Andar,ARQ301,20,"1 Ar condicionado, 1 Projetor multimídia, 1 Me...",1,1,0
...,...,...,...,...,...,...,...
123,NaN,PGEAS2,25,Laboratório,0,0,1
124,NaN,SPL313,30,Laboratório,0,0,1
125,NaN,SPL314,15,Laboratório,0,0,1
126,NaN,SPL315,30,Laboratório,0,0,1


# Lecture required seats fix

In [4]:
# we assume that the actual amount of seats required is at most the amount offered by the class allocated
df_classes['Ocupadas_fix'] = df_classes[['Capacidade', 'Ocupadas']].min(axis='columns')

# Lab rooms+lectures

In [5]:
labs = df_rooms[df_rooms['Laboratório'] > 0]['Sala']

# remove labs
df_rooms = df_rooms[df_rooms['Laboratório'] == 0].drop(columns=['Laboratório'])

print(f"Labs: {len(labs)}")

df_rooms

Labs: 60


,Lugar,Sala,Carteiras,Características,Ar condicionado,Projetor
0,ARQ05 - 1ºAndar,ARQ201,45,"2 Ar condicionado, 1 Mesa do professor (sala d...",2,1
1,ARQ04 - 1ºAndar,ARQ202,40,"1 Ar condicionado, 1 Projetor multimídia, 1 Me...",1,1
2,ARQ03 - 1ºAndar,ARQ203,40,"2 Ar condicionado, 1 Projetor multimídia, 1 Me...",2,1
3,ARQ02 - Térreo,ARQ204,40,"2 Ar condicionado, 1 Projetor multimídia, 1 Qu...",2,1
4,ARQ09 - 2º Andar,ARQ301,20,"1 Ar condicionado, 1 Projetor multimídia, 1 Me...",1,1
...,...,...,...,...,...,...
65,NaN,EQA018,45,"50 Carteira com cadeira separada, 1 Projetor m...",2,1
66,NaN,EQA020,35,"2 Ar condicionado, 35 Carteira com cadeira sep...",2,1
67,NaN,EQA021,35,"2 Ar condicionado, 35 Carteira com cadeira sep...",2,1
68,NaN,EQA022,35,"2 Ar condicionado, 35 Carteira com cadeira sep...",2,1


In [6]:
# remove lectures that occur in labs
df_classes = df_classes[~df_classes['Sala'].isin(labs)]
df_classes = df_classes[df_classes['Sala'] != 'LABCIG']
df_classes

,Bloco,Capacidade,Andar,Sala,Materia,Codigo,Turma,Horario,Oferecidas,Ocupadas,Ocupadas_fix
0,CTC 39 - Bloco A (etapa 1) - Sala de aula Sala...,45,ARQ05 - 1ºAndar,ARQ201,"Historia da Arte, Arquitetura e Urbanismo II",ARQ5622,02207A,2. 0910-3,25,20,20
1,CTC 39 - Bloco A (etapa 1) - Sala de aula Sala...,45,ARQ05 - 1ºAndar,ARQ201,"História da Arte, Arquitetura e Urbanismo III",ARQ5623,04207A,2. 1420-4,25,20,20
2,CTC 39 - Bloco A (etapa 1) - Sala de aula Sala...,45,ARQ05 - 1ºAndar,ARQ201,Arquitetura Latino Americana,ARQ5626,08207A,3. 0910-2,25,11,11
3,CTC 39 - Bloco A (etapa 1) - Sala de aula Sala...,45,ARQ05 - 1ºAndar,ARQ201,Criatividade e Inovação,EGC5027,09207,3. 1420-4,30,11,11
4,CTC 39 - Bloco A (etapa 1) - Sala de aula Sala...,45,ARQ05 - 1ºAndar,ARQ201,Análise Estrutural I,EMC5313,10203,4. 0730-3,20,10,10
...,...,...,...,...,...,...,...,...,...,...,...
1095,EQA023 Sala de aula Sala com carteira 45,45,NaN,EQA023,Operações Unitárias de Transferência de Calor ...,EQA5333,08216,5. 1330-2,20,20,20
1096,EQA023 Sala de aula Sala com carteira 45,45,NaN,EQA023,Fenômenos de Transferência II,EQA5416,06216,5. 1510-2,20,20,20
1097,EQA023 Sala de aula Sala com carteira 45,45,NaN,EQA023,Programação Econômica e Financeira,EPS5211,07216,6. 0730-3,31,26,26
1098,EQA023 Sala de aula Sala com carteira 45,45,NaN,EQA023,Operações Unitárias de Transferência de Quanti...,EQA5313,06215,6. 1330-2,20,15,15


# Room names

In [7]:
df_classes['Sala'].str.replace(' ','').nunique()

65

In [8]:
df_rooms['Sala'].str.replace(' ','').nunique()

68

In [9]:
df_classes['Sala'] = df_classes['Sala'].str.replace(' ', '')
df_rooms['Sala'] = df_rooms['Sala'].str.replace(' ', '')

In [10]:
# rooms in `df_classes` that are not in `df_rooms`
set(df_classes['Sala'].unique()).difference(set(df_rooms['Sala'].unique()))

set()

# ARQ rooms

In [11]:
arq_rooms = df_classes[df_classes['Codigo'].str.contains('ARQ') & \
                       df_classes['Sala'].str.contains('ARQ')]['Sala'].unique()

df_rooms = df_rooms[~df_rooms['Sala'].isin(arq_rooms)]

df_rooms

,Lugar,Sala,Carteiras,Características,Ar condicionado,Projetor
9,ARQ01- 2ºAndar,ARQ308,20,"1 Mesa do professor (sala de aula), 1 Quadro p...",1,1
14,CTC101 - Térreo,CTC101,40,"1 Projetor multimídia, 40 Carteira com cadeira...",2,1
15,CTC102 - Térreo,CTC102,80,"80 Carteira com cadeira separada, 2 Ar condici...",2,1
16,CTC103 - Térreo,CTC103,40,"40 Carteira com cadeira separada, 2 Ar condici...",2,1
17,CTC104 - Térreo,CTC104,35,"2 Ar condicionado, 35 Carteira com cadeira sep...",2,1
18,CTC105 - Térreo,CTC105,40,"2 Ar condicionado, 40 Carteira com cadeira sep...",2,1
19,CTC106 - Térreo,CTC106,40,"2 Ar condicionado, 40 Carteira com cadeira sep...",2,1
20,CTC107 - Térreo,CTC107,40,"2 Ar condicionado, 40 Carteira com cadeira sep...",2,1
21,CTC108 - Térreo,CTC108,40,"2 Ar condicionado, 40 Carteira com cadeira sep...",2,1
22,CTC109 - Térreo,CTC109,35,"2 Ar condicionado, 35 Carteira com cadeira sep...",2,1


In [12]:
df_classes = df_classes[~df_classes['Codigo'].str.contains('ARQ')]

df_classes

,Bloco,Capacidade,Andar,Sala,Materia,Codigo,Turma,Horario,Oferecidas,Ocupadas,Ocupadas_fix
3,CTC 39 - Bloco A (etapa 1) - Sala de aula Sala...,45,ARQ05 - 1ºAndar,ARQ201,Criatividade e Inovação,EGC5027,09207,3. 1420-4,30,11,11
4,CTC 39 - Bloco A (etapa 1) - Sala de aula Sala...,45,ARQ05 - 1ºAndar,ARQ201,Análise Estrutural I,EMC5313,10203,4. 0730-3,20,10,10
7,CTC 39 - Bloco A (etapa 1) - Sala de aula Sala...,45,ARQ05 - 1ºAndar,ARQ201,Química Geral e Inorgânica I,QMC5152,01216,6. 0820-2,40,26,26
9,CTC 39 - Bloco A (etapa 1) - Sala de aula Sala...,40,ARQ04 - 1ºAndar,ARQ202,Instalações Prediais II,ECV5644,05207B,2. 1330-3,33,31,31
11,CTC 39 - Bloco A (etapa 1) - Sala de aula Sala...,40,ARQ04 - 1ºAndar,ARQ202,Estruturas de Concreto,ECV5228,08211,3. 0730-3,30,22,22
...,...,...,...,...,...,...,...,...,...,...,...
1095,EQA023 Sala de aula Sala com carteira 45,45,NaN,EQA023,Operações Unitárias de Transferência de Calor ...,EQA5333,08216,5. 1330-2,20,20,20
1096,EQA023 Sala de aula Sala com carteira 45,45,NaN,EQA023,Fenômenos de Transferência II,EQA5416,06216,5. 1510-2,20,20,20
1097,EQA023 Sala de aula Sala com carteira 45,45,NaN,EQA023,Programação Econômica e Financeira,EPS5211,07216,6. 0730-3,31,26,26
1098,EQA023 Sala de aula Sala com carteira 45,45,NaN,EQA023,Operações Unitárias de Transferência de Quanti...,EQA5313,06215,6. 1330-2,20,15,15


# Course-class naming

In [16]:
df_classes['Turma'].nunique()

286

In [17]:
joint_classes = df_classes.groupby(['Codigo', 'Sala', 'Horario'])['Turma'].nunique()
classes_to_join = df_classes.groupby(['Codigo', 'Sala', 'Horario'])['Turma'].unique()[joint_classes[joint_classes > 1].index]
classes_new_name = df_classes.groupby(['Codigo', 'Sala', 'Horario'])['Turma'].first()[joint_classes[joint_classes > 1].index]
classes_new_name

Codigo   Sala    Horario  
DAS5102  CTC208  5. 1010-2    02220A
DAS5109  EEL008  5. 1620-2    05220A
DAS5120  CTC111  5. 1510-2    06220A
         EEL010  3. 1010-2    06220A
DAS5142  CTC113  6. 1330-2    07220A
                               ...  
MTM3104  CTC102  4. 1620-2     04203
                 6. 1620-2     04203
QMC5222  CTC204  6. 1330-2     02215
QMC5229  CTC304  4. 1010-2     03215
         CTC305  6. 0730-2     03215
Name: Turma, Length: 159, dtype: object

In [18]:
classes_to_join

Codigo   Sala    Horario  
DAS5102  CTC208  5. 1010-2            [02220A, 02220B]
DAS5109  EEL008  5. 1620-2            [05220A, 05220B]
DAS5120  CTC111  5. 1510-2    [06220A, 06220B, 06220C]
         EEL010  3. 1010-2    [06220A, 06220B, 06220C]
DAS5142  CTC113  6. 1330-2    [07220A, 07220B, 07220C]
                                        ...           
MTM3104  CTC102  4. 1620-2              [04203, 04212]
                 6. 1620-2              [04203, 04212]
QMC5222  CTC204  6. 1330-2              [02215, 02216]
QMC5229  CTC304  4. 1010-2              [03215, 03216]
         CTC305  6. 0730-2              [03215, 03216]
Name: Turma, Length: 159, dtype: object

In [19]:
classes_renaming = {c_i: c_o for c_is, c_o in zip(classes_to_join.values, classes_new_name.values) for c_i in c_is}
classes_renaming

{'02220A': '02220A',
 '02220B': '02220A',
 '05220A': '05220A',
 '05220B': '05220A',
 '06220A': '06220A',
 '06220B': '06220A',
 '06220C': '06220A',
 '07220A': '07220A',
 '07220B': '07220A',
 '07220C': '07220A',
 '03220A': '03220A',
 '03220B': '03220A',
 '03220C': '03220A',
 '03220D': '03220A',
 '04220A': '04220A',
 '04220B': '04220A',
 '01220A': '01220A',
 '01220B': '01220A',
 '01220C': '01220A',
 '06201A': '06201A',
 '06201C': '06201A',
 '06212': '06212',
 '06201B': '06201B',
 '06211B': '06201B',
 '05201B': '05201B',
 '05201A': '05201A',
 '05212': '05201A',
 '07211': '06212',
 '09201A': '09201A',
 '09201B': '09201B',
 '07212': '05201B',
 '08201B': '07212',
 '01208B': '01208B',
 '01208D': '01208B',
 '01208A': '01208A',
 '03202A': '03202A',
 '03235': '03202A',
 '08213A': '02220A',
 '06203': '06203',
 '07214': '06203B',
 '06220': '06220',
 '06220F': '06220',
 '01202B': '01202B',
 '01235B': '01202B',
 '01202A': '01202A',
 '01235A': '01202A',
 '04235A': '04202B',
 '04235B': '04202B',
 '0420

In [24]:
df_classes['Turma'] = df_classes['Turma'].map(classes_renaming)
df_classes['Turma'].nunique()

<ipython-input-24-87087d70a669>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_classes['Turma'] = df_classes['Turma'].map(classes_renaming)


68

In [25]:
# drop letters from class
df_classes['Codigo'] = df_classes['Codigo'] + '-' + df_classes['Turma']

<ipython-input-25-978ab8735e6f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_classes['Codigo'] = df_classes['Codigo'] + '-' + df_classes['Turma']


# Course-room matching

In [26]:
# are there multiple classes attending the same lecture at the same time in the same room?
seats_req = df_classes.groupby(['Codigo', 'Horario', 'Sala'])['Ocupadas'].sum()
seats_req

Codigo          Horario    Sala  
BQA5126-06215   2. 1330-2  EQA020    13
                4. 1330-2  EQA020    13
DAS5102-02220A  5. 1010-2  CTC208    36
DAS5104-07220A  2. 1010-2  CTC108     8
DAS5109-05220A  5. 1620-2  EEL008    33
                                     ..
MTM3104-04203   6. 1620-2  CTC102    57
QMC5222-02215   6. 1330-2  CTC204    63
QMC5229-03215   4. 1010-2  CTC304    37
                6. 0730-2  CTC305    37
QMC5350-02215   6. 1010-2  EQA020    16
Name: Ocupadas, Length: 459, dtype: int64

In [27]:
seats_avail = df_classes.groupby(['Codigo', 'Horario', 'Sala'])['Capacidade'].first()
seats_avail

Codigo          Horario    Sala  
BQA5126-06215   2. 1330-2  EQA020    35
                4. 1330-2  EQA020    35
DAS5102-02220A  5. 1010-2  CTC208    60
DAS5104-07220A  2. 1010-2  CTC108    40
DAS5109-05220A  5. 1620-2  EEL008    40
                                     ..
MTM3104-04203   6. 1620-2  CTC102    80
QMC5222-02215   6. 1330-2  CTC204    60
QMC5229-03215   4. 1010-2  CTC304    60
                6. 0730-2  CTC305    60
QMC5350-02215   6. 1010-2  EQA020    35
Name: Capacidade, Length: 459, dtype: int64

In [28]:
# every course-room-time combination is considered a lecture (actualy a long lecture)
df_lectures = df_classes.groupby(['Codigo', 'Horario', 'Sala'])['Ocupadas'].sum().reset_index()
df_lectures['long_lecture'] = df_lectures.index

# each row is a lecture
df_lectures

,Codigo,Horario,Sala,Ocupadas,long_lecture
0,BQA5126-06215,2. 1330-2,EQA020,13,0
1,BQA5126-06215,4. 1330-2,EQA020,13,1
2,DAS5102-02220A,5. 1010-2,CTC208,36,2
3,DAS5104-07220A,2. 1010-2,CTC108,8,3
4,DAS5109-05220A,5. 1620-2,EEL008,33,4
...,...,...,...,...,...
454,MTM3104-04203,6. 1620-2,CTC102,57,454
455,QMC5222-02215,6. 1330-2,CTC204,63,455
456,QMC5229-03215,4. 1010-2,CTC304,37,456
457,QMC5229-03215,6. 0730-2,CTC305,37,457


In [29]:
df_lectures['n_lectures'] = df_lectures['Horario'].str.split('-').apply(lambda x: x[-1])
df_lectures

,Codigo,Horario,Sala,Ocupadas,long_lecture,n_lectures
0,BQA5126-06215,2. 1330-2,EQA020,13,0,2
1,BQA5126-06215,4. 1330-2,EQA020,13,1,2
2,DAS5102-02220A,5. 1010-2,CTC208,36,2,2
3,DAS5104-07220A,2. 1010-2,CTC108,8,3,2
4,DAS5109-05220A,5. 1620-2,EEL008,33,4,2
...,...,...,...,...,...,...
454,MTM3104-04203,6. 1620-2,CTC102,57,454,2
455,QMC5222-02215,6. 1330-2,CTC204,63,455,2
456,QMC5229-03215,4. 1010-2,CTC304,37,456,2
457,QMC5229-03215,6. 0730-2,CTC305,37,457,2


In [30]:
df_lectures = df_lectures.loc[df_lectures.index.repeat(df_lectures['n_lectures'])]
df_lectures = df_lectures.reset_index().drop(columns=['index', 'n_lectures'])
df_lectures['lecture'] = df_lectures.index

df_lectures

,Codigo,Horario,Sala,Ocupadas,long_lecture,lecture
0,BQA5126-06215,2. 1330-2,EQA020,13,0,0
1,BQA5126-06215,2. 1330-2,EQA020,13,0,1
2,BQA5126-06215,4. 1330-2,EQA020,13,1,2
3,BQA5126-06215,4. 1330-2,EQA020,13,1,3
4,DAS5102-02220A,5. 1010-2,CTC208,36,2,4
...,...,...,...,...,...,...
1007,QMC5229-03215,4. 1010-2,CTC304,37,456,1007
1008,QMC5229-03215,6. 0730-2,CTC305,37,457,1008
1009,QMC5229-03215,6. 0730-2,CTC305,37,457,1009
1010,QMC5350-02215,6. 1010-2,EQA020,16,458,1010


# Time periods

In [31]:
days = np.arange(1,7+1)
times = ['0730', '0820', '0910', '1010', '1100', '1330', '1420', '1510', '1620', '1710', '1830', '1920', '2020', '2110']

time_periods = product(days, times)
time_periods = [str(d)+'-'+t for d, t in time_periods]
time_periods

['1-0730',
 '1-0820',
 '1-0910',
 '1-1010',
 '1-1100',
 '1-1330',
 '1-1420',
 '1-1510',
 '1-1620',
 '1-1710',
 '1-1830',
 '1-1920',
 '1-2020',
 '1-2110',
 '2-0730',
 '2-0820',
 '2-0910',
 '2-1010',
 '2-1100',
 '2-1330',
 '2-1420',
 '2-1510',
 '2-1620',
 '2-1710',
 '2-1830',
 '2-1920',
 '2-2020',
 '2-2110',
 '3-0730',
 '3-0820',
 '3-0910',
 '3-1010',
 '3-1100',
 '3-1330',
 '3-1420',
 '3-1510',
 '3-1620',
 '3-1710',
 '3-1830',
 '3-1920',
 '3-2020',
 '3-2110',
 '4-0730',
 '4-0820',
 '4-0910',
 '4-1010',
 '4-1100',
 '4-1330',
 '4-1420',
 '4-1510',
 '4-1620',
 '4-1710',
 '4-1830',
 '4-1920',
 '4-2020',
 '4-2110',
 '5-0730',
 '5-0820',
 '5-0910',
 '5-1010',
 '5-1100',
 '5-1330',
 '5-1420',
 '5-1510',
 '5-1620',
 '5-1710',
 '5-1830',
 '5-1920',
 '5-2020',
 '5-2110',
 '6-0730',
 '6-0820',
 '6-0910',
 '6-1010',
 '6-1100',
 '6-1330',
 '6-1420',
 '6-1510',
 '6-1620',
 '6-1710',
 '6-1830',
 '6-1920',
 '6-2020',
 '6-2110',
 '7-0730',
 '7-0820',
 '7-0910',
 '7-1010',
 '7-1100',
 '7-1330',
 '7-1420',

In [32]:
df_lectures['Horario'] = df_lectures['Horario'].str.split('-').apply(lambda x: x[0])
df_lectures['Horario'] = df_lectures['Horario'].str.replace('. ', '-')

df_lectures

<ipython-input-32-01be645fcdbb>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df_lectures['Horario'] = df_lectures['Horario'].str.replace('. ', '-')


,Codigo,Horario,Sala,Ocupadas,long_lecture,lecture
0,BQA5126-06215,2-1330,EQA020,13,0,0
1,BQA5126-06215,2-1330,EQA020,13,0,1
2,BQA5126-06215,4-1330,EQA020,13,1,2
3,BQA5126-06215,4-1330,EQA020,13,1,3
4,DAS5102-02220A,5-1010,CTC208,36,2,4
...,...,...,...,...,...,...
1007,QMC5229-03215,4-1010,CTC304,37,456,1007
1008,QMC5229-03215,6-0730,CTC305,37,457,1008
1009,QMC5229-03215,6-0730,CTC305,37,457,1009
1010,QMC5350-02215,6-1010,EQA020,16,458,1010


# Formulation

In [33]:
df_lectures = df_lectures.rename({
    'Codigo': 'course',
    'Horario': 'time_period',
    'Sala': 'room',
    'Ocupadas': 'size',
}, axis='columns')

df_lectures

,course,time_period,room,size,long_lecture,lecture
0,BQA5126-06215,2-1330,EQA020,13,0,0
1,BQA5126-06215,2-1330,EQA020,13,0,1
2,BQA5126-06215,4-1330,EQA020,13,1,2
3,BQA5126-06215,4-1330,EQA020,13,1,3
4,DAS5102-02220A,5-1010,CTC208,36,2,4
...,...,...,...,...,...,...
1007,QMC5229-03215,4-1010,CTC304,37,456,1007
1008,QMC5229-03215,6-0730,CTC305,37,457,1008
1009,QMC5229-03215,6-0730,CTC305,37,457,1009
1010,QMC5350-02215,6-1010,EQA020,16,458,1010


In [34]:
df_rooms = df_rooms.rename({
    'Sala': 'room',
    'Carteiras': 'size'
}, axis='columns')

df_rooms

,Lugar,room,size,Características,Ar condicionado,Projetor
9,ARQ01- 2ºAndar,ARQ308,20,"1 Mesa do professor (sala de aula), 1 Quadro p...",1,1
14,CTC101 - Térreo,CTC101,40,"1 Projetor multimídia, 40 Carteira com cadeira...",2,1
15,CTC102 - Térreo,CTC102,80,"80 Carteira com cadeira separada, 2 Ar condici...",2,1
16,CTC103 - Térreo,CTC103,40,"40 Carteira com cadeira separada, 2 Ar condici...",2,1
17,CTC104 - Térreo,CTC104,35,"2 Ar condicionado, 35 Carteira com cadeira sep...",2,1
18,CTC105 - Térreo,CTC105,40,"2 Ar condicionado, 40 Carteira com cadeira sep...",2,1
19,CTC106 - Térreo,CTC106,40,"2 Ar condicionado, 40 Carteira com cadeira sep...",2,1
20,CTC107 - Térreo,CTC107,40,"2 Ar condicionado, 40 Carteira com cadeira sep...",2,1
21,CTC108 - Térreo,CTC108,40,"2 Ar condicionado, 40 Carteira com cadeira sep...",2,1
22,CTC109 - Térreo,CTC109,35,"2 Ar condicionado, 35 Carteira com cadeira sep...",2,1


In [35]:
%%time

# basic sets
C = df_lectures['course'].unique()
T = df_lectures['time_period'].unique()
R = df_rooms['room'].unique()
L = df_lectures['lecture'].unique()


# we define the patterns through the `long_lecture`, which groups lectures
# that span through multiple time periods
long_L = df_lectures.groupby('course')['long_lecture'].unique()
longlecture2lectures = df_lectures.groupby('long_lecture')['lecture'].unique()

P = list()
for c in tqdm(C):
    P_c = list(powerset(long_L[c]))[1:]  # remove empty set
    P_c = [[longlecture2lectures[ll] for ll in p] for p in P_c]
    P_c = [np.concatenate(p) for p in P_c]
    P += P_c
P_idx = np.arange(0,len(P))  # so we can hash it

  0%|          | 0/282 [00:00<?, ?it/s]

CPU times: user 142 ms, sys: 931 µs, total: 142 ms
Wall time: 276 ms


In [36]:
# L_c[c] are the lectures of course c
L_c = df_lectures.groupby('course')['lecture'].unique()

# T_l[l] is the time period in which a lecture happens
T_l = {l: df_lectures.iloc[l]['time_period'] for l in L}

In [37]:
# then, P[c] will have all the patterns that preserve contiguous room stability
patterns_course = [df_lectures.loc[p[0]]['course'] for p in P]
P_c = dict()
for c in tqdm(C):
    P_c[c] = [i for i, c_p in enumerate(patterns_course) if c_p == c]

  0%|          | 0/282 [00:00<?, ?it/s]

In [38]:
# then, P[c] will have all the patterns that preserve contiguous room stability
P_c = dict()
for c in tqdm(C):
    P_c[c] = [i for i, c_p in enumerate(patterns_course) if c_p == 0]

P_l = dict()
for l in tqdm(L):
    P_l[l] = list()
    for p_i in P_idx:
        if l in P[p_i]:
            P_l[l].append(p_i)

  0%|          | 0/282 [00:00<?, ?it/s]

  0%|          | 0/1012 [00:00<?, ?it/s]

In [39]:
# seats required by pattern
size = {p_i: df_lectures.loc[P[p_i]]['size'].max() for p_i in tqdm(P_idx)}

# R[p] are the rooms suitable for pattern p
R_p = {p_i: df_rooms[df_rooms['size'] >= size[p_i]]['room'].unique() for p_i in tqdm(P_idx)}

R_c = {c: list({r for p_i in P_c[c] for r in R_p[p_i]}) for c in C}

  0%|          | 0/772 [00:00<?, ?it/s]

  0%|          | 0/772 [00:00<?, ?it/s]

In [40]:
T_p = {p_i: T_l[l] for p_i in tqdm(P_idx) for l in P[p_i]}

P_rt = dict()
for r in tqdm(R):
    P_rt[r] = dict()
    for t in T:
        P_rt[r][t] = [p_i for p_i in P_idx if r in R_p[p_i] and t in T_p[p_i]]

  0%|          | 0/772 [00:00<?, ?it/s]

  0%|          | 0/56 [00:00<?, ?it/s]

In [41]:
w = [len(P[p_i]) for p_i in P_idx]

In [42]:
data = {
    'C': C,
    'T': T,
    'R': R,
    'L': L,
    'P': P,
    'L_c': L_c,
    'T_l': T_l,
    'P_c': P_c,
    'P_l': P_l,
    'P_rt': P_rt,
    'R_p': R_p,
    'R_c': R_c,
    'w': w,
}

with open('data_small.pkl', 'wb') as f:
    pickle.dump(data, f)

In [43]:
len(P)

772

In [23]:
with open('data_small.pkl', 'rb') as f:
    data = pickle.load(f)

C = data['C']
T = data['T']
R = data['R']
L = data['L']
P = data['P']
L_c = data['L_c']
T_l = data['T_l']
P_c = data['P_c']
P_l = data['P_l']
P_rt = data['P_rt']
R_p = data['R_p']
R_c = data['R_c']
w = data['w']

P_idx = np.arange(0, len(P))

# AMPL

In [51]:
from amplpy import AMPL

ampl = AMPL()

ampl.read('classroom.mod')
# ampl.readData('classroom-example.dat')

In [52]:
ampl.getSet('C').setValues(C.astype(str))
ampl.getSet('T').setValues(T.astype(str))
ampl.getSet('R').setValues(R.astype(str))
ampl.getSet('L').setValues(L.astype(str))
ampl.getSet('P').setValues(P_idx.astype(str))

In [53]:
%%time

def set_indexed(ampl, name, var, idx):
    SS = ampl.getSet(name)
    for i in tqdm(idx):
        try:
            SS[str(i)].setValues(np.array(var[i]).astype(str))
        except RuntimeError as e:
            print(i)
            raise e

set_indexed(ampl, 'L_c', L_c, C)

  0%|          | 0/282 [00:00<?, ?it/s]

CPU times: user 60.6 ms, sys: 20.2 ms, total: 80.8 ms
Wall time: 92.8 ms


In [54]:
set_indexed(ampl, 'R_p', R_p, P_idx)
set_indexed(ampl, 'P_l', P_l, L)

  0%|          | 0/772 [00:00<?, ?it/s]

  0%|          | 0/1012 [00:00<?, ?it/s]

In [64]:
_P_rt = ampl.getSet('P_rt')
for r in tqdm(R):
    for t in T:
        _P_rt[str(r),str(t)].setValues(np.array(P_rt[r][t]).astype(str))

  0%|          | 0/56 [00:00<?, ?it/s]

In [65]:
ampl.param['w'] = [len(p) for p in P]

In [66]:
ampl.exportData('ufsc-instance-small.dat')